# First lets handle all the imports

In [1]:
import numpy as np
import pylab as pl
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objs as go
from tqdm import tqdm

In [2]:
ratings = pd.read_csv('/kaggle/input/movielens-20m-dataset/rating.csv', 
                      parse_dates=['timestamp'])

## Lets work with only 30% of the dataset of ratings for managing the memory

In [3]:
rand_userIds = np.random.choice(ratings['userId'].unique(), 
                                size=int(len(ratings['userId'].unique())*0.3), 
                                replace=False)

ratings = ratings.loc[ratings['userId'].isin(rand_userIds)]

print('There are {} rows of data from {} users'.format(len(ratings), len(rand_userIds)))

There are 6021810 rows of data from 41547 users


In [4]:
ratings

,userId,movieId,rating,timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40
...,...,...,...,...
19999781,138490,3654,4.0,2000-11-30 00:04:15
19999782,138490,3713,4.0,2000-11-30 00:27:30
19999783,138490,3865,1.0,2000-11-30 00:05:32
19999784,138490,3879,3.0,2000-11-30 00:05:32


In [5]:
ratings['rank_latest'] = ratings.groupby(['userId'])['timestamp'] \
                                .rank(method='first', ascending=False)

In [6]:
train_ratings = ratings[ratings['rank_latest'] != 1]
test_ratings = ratings[ratings['rank_latest'] == 1]

# drop columns that we no longer need
train_ratings = train_ratings[['userId', 'movieId', 'rating']]
test_ratings = test_ratings[['userId', 'movieId', 'rating']]

In [7]:
train_ratings.loc[:, 'rating'] = 1

train_ratings.sample(5)

,userId,movieId,rating
15904648,110021,931,1.0
3459213,23565,520,1.0
6198879,42663,928,1.0
11789470,81370,527,1.0
301477,2067,246,1.0


In [8]:
# # Get a list of all movie IDs
# all_movieIds = ratings['movieId'].unique()

# # Placeholders that will hold the training data
# users, items, labels = [], [], []

# # This is the set of items that each user has interaction with
# user_item_set = set(zip(train_ratings['userId'], train_ratings['movieId']))

# # 4:1 ratio of negative to positive samples
# num_negatives = 4

# for (u, i) in tqdm(user_item_set):
#     users.append(u)
#     items.append(i)
#     labels.append(1) # items that the user has interacted with are positive
#     for _ in range(num_negatives):
#         # randomly select an item
#         negative_item = np.random.choice(all_movieIds) 
#         # check that the user has not interacted with this item
#         while (u, negative_item) in user_item_set:
#             negative_item = np.random.choice(all_movieIds)
#         users.append(u)
#         items.append(negative_item)
#         labels.append(0) # items not interacted with are negative

In [9]:
from torch.utils.data import Dataset

In [10]:
class MovieLensTrainDataset(Dataset):
    """MovieLens PyTorch Dataset for Training with GPU Support
    
    Args:
        ratings (pd.DataFrame): Dataframe containing the movie ratings
        all_movieIds (list): List containing all movieIds
        device (torch.device): Device to move tensors to (default: cuda if available)
    """

    def __init__(self, ratings, all_movieIds, device=None):
        if device is None:
            device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        
        self.device = device
        print('Inside initialization of DataLoader, device is set')
        self.users, self.items, self.labels = self.get_dataset(ratings, all_movieIds)
        print('Inside initialization of DataLoader, users,items, labels are all set')

    def __len__(self):
        return len(self.users)
  
    def __getitem__(self, idx):
        return self.users[idx], self.items[idx], self.labels[idx]

    def get_dataset(self, ratings, all_movieIds):
        users, items, labels = [], [], []
        user_item_set = set(zip(ratings['userId'], ratings['movieId']))

        num_negatives = 4
        for u, i in tqdm(user_item_set):
            users.append(u)
            items.append(i)
            labels.append(1)
            for _ in range(num_negatives):
                negative_item = np.random.choice(all_movieIds)
                while (u, negative_item) in user_item_set:
                    negative_item = np.random.choice(all_movieIds)
                users.append(u)
                items.append(negative_item)
                labels.append(0)

        # Use .to() with device for more flexible device handling
        return (
            torch.tensor(users).to(self.device), 
            torch.tensor(items).to(self.device), 
            torch.tensor(labels).to(self.device)
        )

In [11]:
import pytorch_lightning as pl
from torch import nn
import torch
from torch.utils.data import DataLoader

In [12]:
class NCF(pl.LightningModule):
    """ Neural Collaborative Filtering (NCF) with GPU Support
    
    Args:
        num_users (int): Number of unique users
        num_items (int): Number of unique items
        ratings (pd.DataFrame): Dataframe containing the movie ratings for training
        all_movieIds (list): List containing all movieIds (train + test)
    """
    
    def __init__(self, num_users, num_items, ratings, all_movieIds):
        super().__init__()
        # Enable automatic GPU conversion with Lightning
        print('Super is initialized')
        self.user_embedding = nn.Embedding(num_embeddings=num_users, embedding_dim=8)
        self.item_embedding = nn.Embedding(num_embeddings=num_items, embedding_dim=8)
        print('User and item embedding is initialized')
        self.fc1 = nn.Linear(in_features=16, out_features=64)
        self.fc2 = nn.Linear(in_features=64, out_features=32)
        print('FC1 and FC2 linear layers is initialized')
        self.output = nn.Linear(in_features=32, out_features=1)
        print('Sigmoid output is initialized')
        self.ratings = ratings
        self.all_movieIds = all_movieIds

        print('Ratings and all_movieIds are initialized')
        
    def forward(self, user_input, item_input):
        # Pass through embedding layers
        user_embedded = self.user_embedding(user_input)
        item_embedded = self.item_embedding(item_input)

        # Concat the two embedding layers
        vector = torch.cat([user_embedded, item_embedded], dim=-1)

        # Pass through dense layer
        vector = torch.relu(self.fc1(vector))
        vector = torch.relu(self.fc2(vector))

        # Output layer
        pred = torch.sigmoid(self.output(vector))

        return pred
    
    def training_step(self, batch, batch_idx):
        user_input, item_input, labels = batch
        predicted_labels = self(user_input, item_input)
        loss = nn.BCELoss()(predicted_labels, labels.view(-1, 1).float())
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters())

    def train_dataloader(self):
        # Determine the device
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        
        return data.DataLoader(
            MovieLensTrainDataset(self.ratings, self.all_movieIds, device=device),
            batch_size=4096, 
            num_workers=4
        )

In [13]:
def train_model(ratings, train_ratings, max_epochs=5, print_every=50):
    # Determine the device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    num_users = ratings['userId'].max() + 1
    num_items = ratings['movieId'].max() + 1
    
    all_movieIds = ratings['movieId'].unique()

    # Initialize model
    model = NCF(num_users, num_items, train_ratings, all_movieIds)
    model = model.to(device)

    print('Model initialized')

    # Optimizer
    optimizer = torch.optim.Adam(model.parameters())
    
    # Loss function
    criterion = nn.BCELoss()

    print('Optimizer and criterion are now initialized')

    # Training loop with verbose output
    for epoch in range(max_epochs):
        model.train()
        total_loss = 0

        print('Model set to train mode and loss set to 0')
        
        # Create DataLoader
        dataloader = DataLoader(
            MovieLensTrainDataset(train_ratings, all_movieIds), 
            batch_size=4096, 
            shuffle=True
        )

        print('Dataloader is initialized')
        
        print(f"Epoch {epoch+1}/{max_epochs}")
        
        for batch_idx, (users, items, labels) in enumerate(dataloader):
            # Zero the parameter gradients
            optimizer.zero_grad()
            
            # Forward pass
            predicted_labels = model(users, items)
            
            # Compute loss
            loss = criterion(predicted_labels, labels.view(-1, 1).float())
            
            # Backward pass and optimize
            loss.backward()
            optimizer.step()
            
            # Accumulate loss
            total_loss += loss.item()
            
            # Print loss periodically
            if (batch_idx + 1) % print_every == 0:
                print(f"  Batch {batch_idx+1}, Loss: {loss.item():.4f}")
        
        # Print epoch summary
        print(f"Epoch {epoch+1} Average Loss: {total_loss/len(dataloader):.4f}\n")
    
    return model

In [19]:
model = train_model(ratings, train_ratings, max_epochs=5, print_every=50)

Super is initialized
User and item embedding is initialized
FC1 and FC2 linear layers is initialized
Sigmoid output is initialized
Ratings and all_movieIds are initialized
Model initialized
Optimizer and criterion are now initialized
Model set to train mode and loss set to 0
Inside initialization of DataLoader, device is set


100%|██████████| 5980263/5980263 [04:23<00:00, 22727.88it/s]


Inside initialization of DataLoader, users,items, labels are all set
Dataloader is initialized
Epoch 1/1
  Batch 50, Loss: 0.5066
  Batch 100, Loss: 0.4953
  Batch 150, Loss: 0.4968
  Batch 200, Loss: 0.4961
  Batch 250, Loss: 0.4693
  Batch 300, Loss: 0.4817
  Batch 350, Loss: 0.4458
  Batch 400, Loss: 0.4263
  Batch 450, Loss: 0.3910
  Batch 500, Loss: 0.3500
  Batch 550, Loss: 0.3360
  Batch 600, Loss: 0.3071
  Batch 650, Loss: 0.3015
  Batch 700, Loss: 0.2819
  Batch 750, Loss: 0.2717
  Batch 800, Loss: 0.2603
  Batch 850, Loss: 0.2625
  Batch 900, Loss: 0.2673
  Batch 950, Loss: 0.2817
  Batch 1000, Loss: 0.2533
  Batch 1050, Loss: 0.2331
  Batch 1100, Loss: 0.2489
  Batch 1150, Loss: 0.2551
  Batch 1200, Loss: 0.2525
  Batch 1250, Loss: 0.2596
  Batch 1300, Loss: 0.2326
  Batch 1350, Loss: 0.2469
  Batch 1400, Loss: 0.2425
  Batch 1450, Loss: 0.2444
  Batch 1500, Loss: 0.2535
  Batch 1550, Loss: 0.2435
  Batch 1600, Loss: 0.2227
  Batch 1650, Loss: 0.2289
  Batch 1700, Loss: 0.24

In [20]:
# Get a list of all movie IDs
all_movieIds = ratings['movieId'].unique()

# # Placeholders that will hold the training data
# users, items, labels = [], [], []

# # This is the set of items that each user has interaction with
# user_item_set = set(zip(train_ratings['userId'], train_ratings['movieId']))

# # 4:1 ratio of negative to positive samples
# num_negatives = 4

# for (u, i) in tqdm(user_item_set):
#     users.append(u)
#     items.append(i)
#     labels.append(1) # items that the user has interacted with are positive
#     for _ in range(num_negatives):
#         # randomly select an item
#         negative_item = np.random.choice(all_movieIds) 
#         # check that the user has not interacted with this item
#         while (u, negative_item) in user_item_set:
#             negative_item = np.random.choice(all_movieIds)
#         users.append(u)
#         items.append(negative_item)
#         labels.append(0) # items not interacted with are negative

In [26]:
# User-item pairs for testing
test_user_item_set = set(zip(test_ratings['userId'], test_ratings['movieId']))

# Dict of all items that are interacted with by each user
user_interacted_items = ratings.groupby('userId')['movieId'].apply(list).to_dict()

hits = []
for step, (u, i) in enumerate(tqdm(test_user_item_set), start=1):
    interacted_items = user_interacted_items[u]
    not_interacted_items = set(all_movieIds) - set(interacted_items)
    selected_not_interacted = list(np.random.choice(list(not_interacted_items), 99))
    test_items = selected_not_interacted + [i]
    
    predicted_labels = np.squeeze(model(
        torch.tensor([u] * 100).to('cuda'),
        torch.tensor(test_items).to('cuda')
    ).to('cpu').detach().numpy())
    
    top10_items = [test_items[j] for j in np.argsort(predicted_labels)[::-1][:10]]
    
    if i in top10_items:
        hits.append(1)
    else:
        hits.append(0)
    
    # Print the hit ratio every 200 steps
    if step % 2000 == 0:
        current_hit_ratio = np.mean(hits)
        print(f"Step {step}: Current Hit Ratio @10 = {current_hit_ratio:.4f}")

# Final hit ratio
print("The Hit Ratio @ 10 is {:.4f}".format(np.mean(hits)))


  1%|          | 231/41547 [00:01<03:39, 187.92it/s]

Step 200: Current Hit Ratio @10 = 0.7800


  1%|          | 421/41547 [00:02<03:40, 186.33it/s]

Step 400: Current Hit Ratio @10 = 0.7825


  2%|▏         | 632/41547 [00:03<03:40, 185.84it/s]

...

Step 41200: Current Hit Ratio @10 = 0.8016


100%|█████████▉| 41436/41547 [03:43<00:00, 183.47it/s]

Step 41400: Current Hit Ratio @10 = 0.8017


100%|██████████| 41547/41547 [03:43<00:00, 185.72it/s]

The Hit Ratio @ 10 is 0.8016


In [28]:
torch.save(model.state_dict(), '/kaggle/working/model_1.pth')